### Imports

In [13]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from sklearn.metrics import roc_curve, auc

### Recolha de dados

In [ ]:
# 64 patients with breast cancer and 52 healthy controls, 116 in total
# 1 - paciente sem cancro, 2 - paciente com cancro

file = pd.read_excel("../Data/dataR2.XLSX")
file = file.sample(frac=1).reset_index(drop=True) #Shuffle data
DataSet = pd.DataFrame(data=file.dropna()) #Remove NaN because of initial empty lines

# Find the indexes where the differente class samples are located
ixHealthy = np.where(DataSet['Classification'] == 1)
ixWithCancer = np.where(DataSet['Classification'] == 2)

# Define classes
Classes = ["healthy","withCancer"]

DataSet['Classification'].iloc[ixHealthy] = Classes[0]
DataSet['Classification'].iloc[ixWithCancer] = Classes[1] 

count = 0

for w in DataSet.columns:
    for i in DataSet.columns: 
        for n in DataSet.columns:
            if count > 10:
                x = input("Press Enter to continue or 'q' to quit: ")
                count = 0
                if x.lower() == 'q':
                    break

            if w == 'Classification' or i == 'Classification' or n == 'Classification' or w == i or w == n or i == n:
                continue
            fig = px.scatter_3d(x = DataSet[n], y = DataSet[w], z = DataSet[i], color = DataSet['Classification'], labels=dict(x = n, y = w, z = i , color = "Classification"))
            count += 1

            fig.update_traces(marker_size=10)
            fig.show()

/var/folders/dc/pvd7drj923v5htn8s380bwbh0000gn/T/ipykernel_85914/3905254520.py:15: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/var/folders/dc/pvd7drj923v5htn8s380bwbh0000gn/T/ipykernel_85914/3905254520.py:15: SettingWithCopyWarning:


A v